In [ ]:
import data_util
from tqdm import tqdm_notebook, tqdm

In [ ]:
import nltk
import re
import time

In [ ]:
def is_punct(s):
    for ch in s.lower():
        i = ord(ch)
        if 97 <= i <= 122 or 48 <= i <= 57:
            return False
    return True

from unidecode import unidecode

def generate_tokens(dataset, partition):
    texts, labels = data_util.load_dataset_csv_raw('dataset/{}/{}.csv'.format(dataset, partition))

    with open('dataset/{}/{}.tokenized.txt'.format(dataset, partition), 'w') as fo:            
        for text in tqdm_notebook(texts):     
            if isinstance(text, unicode):
                decoded_text = text
            else:
                decoded_text = text.decode('utf8')
            for sent in nltk.tokenize.sent_tokenize( unidecode( decoded_text ) ):
                tokens = [ token.lower() for token in nltk.word_tokenize(sent) ]
                tokens[:] = [ token for token in tokens if not is_punct(token) ]
                fo.write(' '.join(tokens) + '\n')

In [ ]:
# change this to where you install the kenlm
KENLM_PATH = "../language_model/kenlm/build/bin"

In [ ]:
generate_tokens('yelp_review_polarity_csv', 'train')
generate_tokens('amazon_review_polarity_csv', 'train')
generate_tokens('aclImdb', 'train')
generate_tokens('trec07p', 'train')
for dataset_name in ['aclImdb', 'trec07p', 'yelp_review_polarity_csv', 'amazon_review_polarity_csv']:
    !{KENLM_PATH}/lmplz -o 4 -S 10G  --text dataset/{dataset_name}/train.tokenized.txt --arpa dataset/{dataset_name}/lm.arpa
    !{KENLM_PATH}/build_binary dataset/{dataset_name}/lm.arpa dataset/{dataset_name}lm.arpa.bin